In [40]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [ ]:
def get_yahoo_html(symbol):
    url = f"https://finance.yahoo.com/quote/{symbol}/analysis"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Print out the entire HTML content of the page
        print(soup.prettify())
    else:
        print("Failed to retrieve data from Yahoo Finance.")

# Example usage:
symbol = 'ADBE'  # Adobe Inc.
get_yahoo_html(symbol)


In [19]:

def get_analyst_price_targets(symbol):
    url = f"https://finance.yahoo.com/quote/{symbol}/analysis"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)

    # Add a delay of 1 second before making the next request
    time.sleep(1)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find the section containing analyst price targets
        target_section = soup.find('section', {'data-testid': 'analyst-price-target-card'})
        if target_section:
            # Extract the high, average, and low price targets
            high_target = target_section.find('div', class_='highLabel').span.text.strip()
            average_target = target_section.find('div', class_='average').find('span', class_='price').text.strip()
            low_target = target_section.find('div', class_='lowLabel').span.text.strip()
            # Create a DataFrame
            df = pd.DataFrame({
                'Target': ['High', 'Average', 'Low'],
                'Price Target': [high_target, average_target, low_target]
            })
            return df
        else:
            print("Analyst price target section not found.")
            return None
    else:
        print("Failed to retrieve data from Yahoo Finance.")
        return None

# Example usage:
symbol = 'ADBE'  # Adobe Inc.
analyst_targets_df = get_analyst_price_targets(symbol)
if analyst_targets_df is not None:
    print("Analyst Price Targets for", symbol)
    display(analyst_targets_df)

Analyst Price Targets for ADBE


,Target,Price Target
0,High,732.52
1,Average,619.28
2,Low,445.00


In [17]:
def get_upgrades_downgrades(symbol):
    url = f"https://finance.yahoo.com/quote/{symbol}/analysis"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Find the section containing upgrades and downgrades
        upgrade_downgrade_section = soup.find('section', {'data-testid': 'upgrade-downgrade-table'})
        if upgrade_downgrade_section:
            # Extract upgrades and downgrades
            table_rows = upgrade_downgrade_section.find_all('tr')
            upgrades_downgrades = []
            for row in table_rows:
                cells = row.find_all('td')
                if len(cells) == 3:
                    action = cells[0].text.strip()
                    recommendation = cells[1].text.strip()
                    date = cells[2].text.strip()
                    upgrades_downgrades.append({'Action': action, 'Recommendation': recommendation, 'Date': date})
            return upgrades_downgrades
        else:
            print("Upgrades and downgrades section not found.")
    else:
        print("Failed to retrieve data from Yahoo Finance.")

# Example usage:
symbol = 'ADBE'  # Adobe Inc.
upgrades_downgrades = get_upgrades_downgrades(symbol)
if upgrades_downgrades:
    # Convert to DataFrame
    df = pd.DataFrame(upgrades_downgrades)
    print("DataFrame:")
    display(df)

DataFrame:


,Action,Recommendation,Date
0,Maintains,DA Davidson: Buy to Buy,3/28/2024
1,Reiterates,JMP Securities: Market Perform,3/28/2024
2,Reiterates,Mizuho: Buy to Buy,3/27/2024
3,Reiterates,RBC Capital: Outperform to Outperform,3/27/2024
4,Maintains,Evercore ISI Group: Outperform to Outperform,3/27/2024
5,Maintains,BMO Capital: Outperform to Outperform,3/27/2024
